In [ ]:
def detect_faces_mtcnn(img, img_index, cropped_faces):
    try:
        correct_rotation = 0
        correct_box_length = []
        highest_prob = 0.0 
        for i in range(4):
            empty_box = False
            degrees = 90 * i
            rotated_img = img
            if degrees == 90:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif degrees == 180:
                rotated_img = cv2.rotate(img, cv2.ROTATE_180)
            elif degrees == 270:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            else:
                pass
            
            height, width, _ = rotated_img.shape
            #resized_img = cv2.resize(img, (width // 6, height // 6))
            resized_img = cv2.resize(rotated_img, (width, height))
            confidence_threshold = 0.83
            boxes, probs = mtcnn.detect(resized_img)
            rotated_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)

            if boxes is None or probs is None:
                empty_box = True
                boxes = []

            #print(boxes)
            print(probs)

            if empty_box == False:
                boxes = [box for box, prob in zip(boxes, probs) if prob >= confidence_threshold]
                boxes = sorted(boxes, key=lambda box: box[0])

                for box in boxes:
                    for prob in probs:
                        if prob > highest_prob:
                            highest_prob = prob
                            correct_box_length = boxes
                            correct_rotation = degrees
                    #print(len(correct_box_length))
                    x1, y1, x2, y2 = [int(coord) for coord in box]

                    #x1, y1, x2, y2 = x1 * 6, y1 * 6, x2 * 6, y2 * 6
                    height, width, _ = rotated_img.shape

                    x1 = max(0, min(x1, width - 1))
                    y1 = max(0, min(y1, height - 1))
                    x2 = max(0, min(x2, width - 1))
                    y2 = max(0, min(y2, height - 1))

                    cropped_face = rotated_img[y1:y2, x1:x2]
                    cropped_faces.append(cropped_face)

                    cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)
                    if len(cropped_face.shape) == 3:
                        plt.imshow(cropped_face)
                        plt.axis('off')
                        plt.show()
                    else:
                        plt.imshow(cropped_face, cmap='gray')
                        plt.axis('off')
                        plt.show()
                    
                    cv2.rectangle(rotated_img, (x1, y1), (x2, y2), (0, 255, 0), 2)


        if correct_rotation == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif correct_rotation == 180:
            img = cv2.rotate(img, cv2.ROTATE_180)
        elif correct_rotation == 270:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            pass
        
        if len(img.shape) == 3:
            plt.imshow(img)
            plt.axis('off')
            plt.show()
        else:
            plt.imshow(img, cmap='gray')
            plt.axis('off')
            plt.show()

        Cropped_FilePath = "C:/Users/Senne/AI_Frameworks/Project/CroppedTestImg"
        os.makedirs(Cropped_FilePath, exist_ok=True)

        return len(correct_box_length)
    except Exception as e:
        print(f"Error detecting faces in image index {img_index}: {e}")
        return 0

In [ ]:
def Valid_Landmarks(box, Landmark):
    hight = box[3] - box[1]
    width = box[2] - box[0]
    left_eye, right_eye, nose, left_mouth, right_mouth = Landmark

    eye_distance = np.linalg.norm(np.array(left_eye) - np.array(right_eye))
    nose_to_mouth_distance = np.linalg.norm(np.array(nose) - (np.array(left_mouth) + np.array(right_mouth)) / 2)

    #print(f"eye distance: {eye_distance}")
    #print(f"nose to mouth distance: {nose_to_mouth_distance}")

    if eye_distance < 10:
        return False
    
    return True

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

recognizer.read("C:/Users/Senne/AI_Frameworks/Project/Models/Trainer_LBHF.yml")
dir = "C:/Users/Senne/AI_Frameworks/Project/CroppedTestImg"

results_FileNames = []


desired_width = 500
desired_height = 550

P_name = {"personName": 1}
with open("Labels_faces.pickle", 'rb') as f:
    P_name = pickle.load(f)
    P_name = {n:i for i,n in P_name.items()}

total = 0

results = pd.DataFrame()
results = pd.concat([results, bad_detect], ignore_index=True)

for image_file in os.listdir(dir):
    Images = []
    total += 1
    img_path = os.path.join(dir, image_file)


    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    id_, conf = recognizer.predict(img)

    print(P_name.get(id_))
    print(conf)


    if conf >= 45 or conf <= 85:
        predicted_label = P_name.get(id_, "Unknown")
    
    results = pd.concat([results, pd.DataFrame({
        'image': [img_path],
        'guess': [predicted_label]
    })], ignore_index=True)

    if total % 25 == 0:
        print(f"Processed {total} images")

results = results.sort_values(by='image', key=lambda col: col.map(os.path.basename)).reset_index(drop=True)

In [ ]:
df_embeddings = pd.read_pickle("C:/Users/Senne/AI_Frameworks/Project/embeddings.pkl")

with open("Labels_faces.pickle", 'rb') as f:
    P_name = pickle.load(f)
    P_name = {n: i for i, n in P_name.items()}

dir = "C:/Users/Senne/AI_Frameworks/Project/CroppedTestImg"

shape_predictor_path = "C:/Users/Senne/AI_Frameworks/Project/Models/shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "C:/Users/Senne/AI_Frameworks/Project/Models/dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
side_detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(shape_predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

results = pd.DataFrame()

for image_file in os.listdir(dir):
    img_path = os.path.join(dir, image_file)

    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    dets = detector(img_rgb, 1)
    
    if len(dets) == 0:
        dets = side_detector(img_rgb, 1)
    
    if len(dets) == 0:
        continue

    for k, d in enumerate(dets):
        shape = sp(img_rgb, d)
        
        face_chip = dlib.get_face_chip(img_rgb, shape)

        face_descriptor = facerec.compute_face_descriptor(face_chip)

        embedding = np.array(face_descriptor).reshape(1, -1)

        similarities = []
        for stored_embedding in df_embeddings['embedding']:
            stored_embedding = np.array(stored_embedding).reshape(1, -1)
            similarity = cosine_similarity(embedding, stored_embedding)
            similarities.append(similarity[0][0])

        max_similarity_index = np.argmax(similarities)
        best_match_id = df_embeddings.iloc[max_similarity_index]['id']
        predicted_label = P_name.get(best_match_id, "Unknown")
        confidence = similarities[max_similarity_index]

        results = pd.concat([results, pd.DataFrame({
            'image': [img_path],
            'guess': [predicted_label],
            'confidence': confidence,
        })], ignore_index=True)

        print(f"Processed: {image_file} - Predicted: {predicted_label} - Similarity: {confidence:.4f}")

results = results.sort_values(by='image', key=lambda col: col.map(os.path.basename)).reset_index(drop=True)

print(results)

In [ ]:
test_data_FileNames = []
results_FileNames = []

for i in range(len(test_data)):
    person_path = test_data[i]
    File_name = os.path.splitext(os.path.basename(person_path))[0]
    File_name = File_name.lstrip('0')
    File_name = File_name.split('_')[0]
    test_data_FileNames.append(File_name)

for i in range(len(results)):
    person_path = results['image'][i]
    File_name = os.path.splitext(os.path.basename(person_path))[0]
    File_name = File_name.lstrip('0')
    File_name = File_name.split('_')[0]
    results_FileNames.append(File_name)